## Reading EuXFEL bunch pattern data

Each pulse train at EuXFEL consists of up to 2700 X-ray pulses.
The 'bunch pattern table' stores information about the production of pulses,
each of which corresponds to one bunch of electrons in the accelerator.

The information for each pulse is packed into a 32-bit integer.
Every table has 2700 entries - skipped pulses are represented as 0.

The code below loads the bunch tables for all trains in the specified run.

In [1]:
from extra_data import open_run
import euxfel_bunch_pattern

In [2]:
# Proposal 700000 contains sample data - this run is originally from prop. 2212 at SCS.
run = open_run(proposal=700000, run=26)

bunch_pattern_tables = run.get_array(
    'SCS_RR_UTC/TSYS/TIMESERVER', 'bunchPatternTable.value', extra_dims=['pulse_slot']
)

bunch_pattern_tables[:5]   # Examine the first few tables

<xarray.DataArray (trainId: 5, pulse_slot: 2700)>
array([[2141993,       0, 2129961, ...,       0,       0,       0],
       [2139945,       0, 2129961, ...,       0,       0,       0],
       [2141993,       0, 2129961, ...,       0,       0,       0],
       [2139945,       0, 2129961, ...,       0,       0,       0],
       [2141993,       0, 2129961, ...,       0,       0,       0]],
      dtype=uint32)
Coordinates:
  * trainId  (trainId) uint64 517755296 517755297 517755298 517755299 517755300
Dimensions without coordinates: pulse_slot

## Destinations

The 32 bits of each entry record different information, including the destination of the pulse.

E.g. we can see which pulses in each train were sent to different SASEs:

In [3]:
for table in bunch_pattern_tables[:5]:
    tid = table.trainId.item()
    sase1_pulses = euxfel_bunch_pattern.indices_at_sase(table.data, sase=1)
    sase3_pulses = euxfel_bunch_pattern.indices_at_sase(table.data, sase=3)
    print(f"Train {tid}: {len(sase1_pulses)} pulses to SASE1, {len(sase3_pulses)} to SASE3")

Train 517755296: 10 pulses to SASE1, 50 to SASE3
Train 517755297: 10 pulses to SASE1, 50 to SASE3
Train 517755298: 10 pulses to SASE1, 50 to SASE3
Train 517755299: 10 pulses to SASE1, 50 to SASE3
Train 517755300: 10 pulses to SASE1, 50 to SASE3


The `BUNCH_DECODER` source (Karabo device `pulsePatternDecoder`) records this information in a more convenient format:

In [4]:
run.get_array('SCS_RR_UTC/MDL/BUNCH_DECODER', 'sase3.nPulses.value')[:10]

<xarray.DataArray (trainId: 10)>
array([50, 50, 50, 50, 50, 50, 50, 50, 50, 50], dtype=int32)
Coordinates:
  * trainId  (trainId) uint64 517755296 517755297 ... 517755304 517755305

We can also check this against an *X-ray Gas Monitor* (XGM) in SASE3.

In the numbers below, an XGM intensity of ~6000 indicates a pulse in SASE3.
When there's no pulse, the intensity value is much smaller.

The XGM may not record data for all trains, so we need to align the data.
In the output below, missing data from the XGM shows up as `nan` (Not A Number).
If we aligned with `join='inner'`, these rows would be dropped instead.

In [5]:
import numpy as np
import xarray
np.set_printoptions(precision=1, suppress=True)  # Simplify number formatting

In [6]:
sa3_xgm_intensity = run.get_array('SA3_XTD10_XGM/XGM/DOOCS:output', 'data.intensityTD')

bunch_pattern_tables, sa3_xgm_intensity = \
    xarray.align(bunch_pattern_tables, sa3_xgm_intensity, join='outer')

for bunch_table, intensities in zip(bunch_pattern_tables[:10], sa3_xgm_intensity):
    print(f"Train {bunch_table.trainId.item()}:")
    sa3_pulses = euxfel_bunch_pattern.indices_at_sase(bunch_table.values, sase=3)
    print(f"  Bunches to SASE3: {sa3_pulses[:10]}...")
    print(f"   XGM intensities: {intensities[:10].values}...")
    print()

Train 517755296:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [   4.2 7244.2  -13.6   -1.2 7265.8  -19.6    2.7 6235.8   -7.8    5.8]...

Train 517755297:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [  -1.5 6630.   -23.1   -1.3 6854.6  -19.3    9.  6851.4  -22.3  -10. ]...

Train 517755298:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [  10.  6929.4  -15.9    7.4 6962.8  -12.3    2.3 6698.6  -16.8    3.5]...

Train 517755299:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [   0.2 7186.3  -21.1    6.2 7004.5    1.2    7.4 7183.2   -8.3    4.9]...

Train 517755300:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [   5.3 7398.2   -2.9   10.6 6416.2   -9.5    6.  6963.9  -23.2    2.2]...

Train 517755301:
  Bunches to SASE3: [882 890 898 906 914 922 930 938 946 954]...
   XGM intensities: [   2.8 

## Pulse charge

You can also get the pulse charge in nanocoulombs (nC):

In [7]:
euxfel_bunch_pattern.get_charge(bunch_pattern_tables[0, :200])

array([0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. ,
       0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4,
       0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.4, 0. , 0.